In [1]:
import re

def load_sparse_pauli_file(filename):
    with open(filename, 'r') as f:
        content = f.read()

    # Step 1: Extract the Pauli strings inside SparsePauliOp([...])
    match = re.search(r"SparsePauliOp\(\[([^\]]+)\]", content)
    if not match:
        raise ValueError("Could not find Pauli strings in file.")

    raw_strings = match.group(1).split(',')
    pauli_strings = [s.strip().strip("'").replace(" ", "") for s in raw_strings if s.strip()]

    return pauli_strings

In [2]:
def maximal_munch(pauli_str):
    chunks = []
    start = None

    for i, c in enumerate(pauli_str):
        if c in {'X', 'Y', 'Z'}:
            if start is None:
                start = i
        else:
            if start is not None:
                chunks.append((start, pauli_str[start:i]))
                start = None

    if start is not None:
        chunks.append((start, pauli_str[start:]))

    return chunks


In [3]:
def munch_file(filename):
    pauli_strings = load_sparse_pauli_file(filename)
    all_chunks = [(pstr, maximal_munch(pstr)) for pstr in pauli_strings]
    return all_chunks

In [4]:
for pauli, munches in munch_file("hamiltonian_pauli.txt"):
    print(f"{pauli} → {munches}")

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [5]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, Pauli, Operator
import numpy as np

def apply_pauli_string(pauli_str, state):
    """Applies a Pauli string like 'XIZ' to a statevector."""
    n = len(pauli_str)
    qc = QuantumCircuit(n)
    for i, p in enumerate(reversed(pauli_str)):  # Qiskit is little-endian
        if p == 'X':
            qc.x(i)
        elif p == 'Y':
            qc.y(i)
        elif p == 'Z':
            qc.z(i)
    return state.evolve(Operator(qc))

def states_equal_up_to_global_phase(state1, state2, tol=1e-10):
    """Returns True if state1 == e^{iφ} * state2"""
    inner_product = np.vdot(state1.data, state2.data)
    return np.abs(np.abs(inner_product) - 1) < tol

original_ops = ['XII', 'IXI', 'IIX']
munched_op = 'XXX'

init = Statevector.from_label('000')

state_U = init
for op in original_ops:
    state_U = apply_pauli_string(op, state_U)

state_V = apply_pauli_string(munched_op, init)

print("States equal:", states_equal_up_to_global_phase(state_U, state_V))


States equal: True


In [6]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, Operator
from functools import reduce
import numpy as np

# ------------------------------
# Pauli multiplication table (with phase tracking)
# ------------------------------

pauli_mult_table = {
    ('I', 'I'): ('I', 1),
    ('I', 'X'): ('X', 1),
    ('I', 'Y'): ('Y', 1),
    ('I', 'Z'): ('Z', 1),

    ('X', 'I'): ('X', 1),
    ('X', 'X'): ('I', 1),
    ('X', 'Y'): ('Z', 1j),
    ('X', 'Z'): ('Y', -1j),

    ('Y', 'I'): ('Y', 1),
    ('Y', 'X'): ('Z', -1j),
    ('Y', 'Y'): ('I', 1),
    ('Y', 'Z'): ('X', 1j),

    ('Z', 'I'): ('Z', 1),
    ('Z', 'X'): ('Y', 1j),
    ('Z', 'Y'): ('X', -1j),
    ('Z', 'Z'): ('I', 1),
}

def multiply_pauli_strings(p1, p2):
    """Multiplies two Pauli strings with phase tracking."""
    if len(p1) != len(p2):
        raise ValueError("Pauli strings must be the same length")
    
    result = []
    phase = 1
    for a, b in zip(p1, p2):
        r, p = pauli_mult_table[(a, b)]
        result.append(r)
        phase *= p

    return ''.join(result), phase

def munch_with_phase(pauli_list):
    """Multiplies a list of Pauli strings, tracking global phase."""
    return reduce(lambda acc, p: multiply_pauli_strings(acc[0], p), pauli_list, ('I' * len(pauli_list[0]), 1))


# ------------------------------
# Apply Pauli to a statevector
# ------------------------------

def apply_pauli_string(pauli_str, state):
    """Applies a Pauli string (like 'XIZ') to a statevector."""
    n = len(pauli_str)
    qc = QuantumCircuit(n)
    for i, p in enumerate(reversed(pauli_str)):  # Qiskit is little-endian
        if p == 'X':
            qc.x(i)
        elif p == 'Y':
            qc.y(i)
        elif p == 'Z':
            qc.z(i)
    return state.evolve(Operator(qc))

def apply_pauli_with_phase(pauli_str, phase, state):
    """Applies a Pauli string + global phase to a state."""
    state = apply_pauli_string(pauli_str, state)
    return state * phase

# ------------------------------
# State comparison
# ------------------------------

def states_equal_up_to_global_phase(state1, state2, tol=1e-10):
    """Returns True if state1 == e^{iφ} * state2"""
    inner_product = np.vdot(state1.data, state2.data)
    return np.abs(np.abs(inner_product) - 1) < tol


# ------------------------------
# Commutation check (optional)
# ------------------------------

def pauli_commute(p1, p2):
    """Checks whether two Pauli strings commute."""
    if len(p1) != len(p2):
        raise ValueError("Pauli strings must be the same length")

    anti_commuting_pairs = 0
    for a, b in zip(p1, p2):
        if a == 'I' or b == 'I':
            continue
        elif a == b:
            continue
        elif {a, b} in [{'X', 'Y'}, {'Y', 'Z'}, {'Z', 'X'}, {'X', 'Z'}, {'Z', 'Y'}, {'Y', 'X'}]:
            anti_commuting_pairs += 1

    return anti_commuting_pairs % 2 == 0


# ------------------------------
# Example tests
# ------------------------------

if __name__ == "__main__":
    print("=== Basic examples ===")
    ops = ['XII', 'IXI', 'IIX']  # → XXX, phase = 1
    result, phase = munch_with_phase(ops)
    print(f"ops: {ops} → {result}, phase: {phase}")

    ops2 = ['XII', 'ZII']  # → YII, phase = i
    result2, phase2 = munch_with_phase(ops2)
    print(f"ops: {ops2} → {result2}, phase: {phase2}")

    ops3 = ['XII', 'IXY', 'YIZ']  # a more complex mix
    result3, phase3 = munch_with_phase(ops3)
    print(f"ops: {ops3} → {result3}, phase: {phase3}")

    # Optional: check that resulting operator acts the same
    from qiskit.quantum_info import Statevector

    print("\n=== Verify state equivalence ===")
    init = Statevector.from_label('000')

    original = init
    for op in ops:
        original = apply_pauli_string(op, original)

    combined = apply_pauli_with_phase(result, phase, Statevector.from_label('000'))

    print("States equal up to global phase?", states_equal_up_to_global_phase(original, combined))


=== Basic examples ===
ops: ['XII', 'IXI', 'IIX'] → XXX, phase: 1
ops: ['XII', 'ZII'] → YII, phase: -1j
ops: ['XII', 'IXY', 'YIZ'] → ZXX, phase: (-1+0j)

=== Verify state equivalence ===
States equal up to global phase? True
